# Counter Strike Global Offensive Match Result Prediction
- **Leonardo Valerio Morales 771030**
- **Luis Felipe Dobner Henriques 771036**

This notebook executes data pre-processing and predictive analysis of Counter Strike Global Offensive Matches.

# Enviroment Variables
This step loads Everything needed for Neo4j and Apache Kudu to Work

In [1]:
#Enviroment Variables
import random
import os
from os import listdir

import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F

from ipywidgets import interact, widgets

KUDU_MASTER = 'kudu-master-1:7051'

In [3]:
#Enviroment Variables
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.kudu:kudu-spark3_2.12:1.13.0.7.1.5.17-1,org.neo4j:neo4j-connector-apache-spark_2.12:5.0.1_for_spark_3 --repositories https://repository.cloudera.com/artifactory/cloudera-repos/ pyspark-shell'

In [4]:
#Enviroment Variables
spark = SparkSession.builder.config('spark.packages', 'org.apache.kudu:kudu-spark3_2.12:1.13.0.7.1.5.17-1,org.neo4j:neo4j-connector-apache-spark_2.12:5.0.1_for_spark_3').getOrCreate()
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

# Data Loading
This step will create the tables and schemas in both databases and load the concerning data from the dataset into Neo4j and Apache Kudu.

In [5]:
!unzip -n {'./archive.zip'}

Archive:  ./archive.zip
  inflating: economy.csv             
  inflating: picks.csv               
  inflating: players.csv             
  inflating: results.csv             


In [8]:
# Data Extraction
df1 = spark.read.format("csv").option("header", "true").load("./results.csv")
display(df1)
df1.show(truncate=False)

DataFrame[date: string, team_1: string, team_2: string, _map: string, result_1: string, result_2: string, map_winner: string, starting_ct: string, ct_1: string, t_2: string, t_1: string, ct_2: string, event_id: string, match_id: string, rank_1: string, rank_2: string, map_wins_1: string, map_wins_2: string, match_winner: string]

+----------+--------------------+---------------+--------+--------+--------+----------+-----------+----+---+---+----+--------+--------+------+------+----------+----------+------------+
|date      |team_1              |team_2         |_map    |result_1|result_2|map_winner|starting_ct|ct_1|t_2|t_1|ct_2|event_id|match_id|rank_1|rank_2|map_wins_1|map_wins_2|match_winner|
+----------+--------------------+---------------+--------+--------+--------+----------+-----------+----+---+---+----+--------+--------+------+------+----------+----------+------------+
|2020-03-18|Recon 5             |TeamOne        |Dust2   |0       |16      |2         |2          |0   |1  |0  |15  |5151    |2340454 |62    |63    |0         |2         |2           |
|2020-03-18|Recon 5             |TeamOne        |Inferno |13      |16      |2         |2          |8   |6  |5  |10  |5151    |2340454 |62    |63    |0         |2         |2           |
|2020-03-18|New England Whalers |Station7       |Inferno |12      |16      

In [9]:
# Data Extraction
df2 = spark.read.format("csv") \
.option("header", "true") \
.load("./economy.csv")
display(df2)
df2.show(truncate=False)

DataFrame[date: string, match_id: string, event_id: string, team_1: string, team_2: string, best_of: string, _map: string, t1_start: string, t2_start: string, 1_t1: string, 2_t1: string, 3_t1: string, 4_t1: string, 5_t1: string, 6_t1: string, 7_t1: string, 8_t1: string, 9_t1: string, 10_t1: string, 11_t1: string, 12_t1: string, 13_t1: string, 14_t1: string, 15_t1: string, 16_t1: string, 17_t1: string, 18_t1: string, 19_t1: string, 20_t1: string, 21_t1: string, 22_t1: string, 23_t1: string, 24_t1: string, 25_t1: string, 26_t1: string, 27_t1: string, 28_t1: string, 29_t1: string, 30_t1: string, 1_t2: string, 2_t2: string, 3_t2: string, 4_t2: string, 5_t2: string, 6_t2: string, 7_t2: string, 8_t2: string, 9_t2: string, 10_t2: string, 11_t2: string, 12_t2: string, 13_t2: string, 14_t2: string, 15_t2: string, 16_t2: string, 17_t2: string, 18_t2: string, 19_t2: string, 20_t2: string, 21_t2: string, 22_t2: string, 23_t2: string, 24_t2: string, 25_t2: string, 26_t2: string, 27_t2: string, 28_t

+----------+--------+--------+-------------+-------------+-------+--------+--------+--------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|date      |match_id|event_id|team_1       |team_2       |best_of|_map    |t1_start|t2_start|1_t1  |2_t1   |3_t1   |4_t1   |5_t1   |6_t1   

In [10]:
df_limited = df1.select(df1['match_id'], df1['_map'], df1['team_1'], df1['team_2'], df1['map_winner'], df1['starting_ct'], df1['match_winner'])
display(df_limited)

DataFrame[match_id: string, _map: string, team_1: string, team_2: string, map_winner: string, starting_ct: string, match_winner: string]

In [11]:
df2_limited = df2.select(df2['match_id'], df2['_map'], df2['team_1'], df2['team_2'], df2['best_of'])
display(df2_limited)

DataFrame[match_id: string, _map: string, team_1: string, team_2: string, best_of: string]

In [12]:
#Pré-processamento para Tabela de Jogos no Kudu
df_collect = df_limited.collect()
returnval = []
for i in range(df_limited.count()):
    selected_row = df_collect[i]

    map_winner_num = selected_row['map_winner']
    map_winner = selected_row[f'team_{map_winner_num}']

    starting_ct_num = selected_row['starting_ct']
    start_ct = selected_row[f'team_{starting_ct_num}']

    tr = ''
    if start_ct == selected_row['team_2']:
        tr = selected_row['team_1']
    else:
        tr = selected_row['team_2']
        
    
    returnval.append([selected_row['match_id'], selected_row['_map'], selected_row['team_1'], selected_row['team_2'], map_winner, start_ct, tr])
    

schema = ['partida','mapa', 'equipe1','equipe2','vitorioso','ct','tr']
df_mapas = spark.createDataFrame(returnval, schema)
display(df_mapas)

DataFrame[partida: string, mapa: string, equipe1: string, equipe2: string, vitorioso: string, ct: string, tr: string]

In [15]:
def store_in_kudu(df, table):
    df.write.option('kudu.master', KUDU_MASTER).option('kudu.table', f'impala::default.{table}').mode('append').format('kudu').save()

In [20]:
def read_from_kudu(table):
     return spark.read.option('kudu.master', KUDU_MASTER).option('kudu.table', f'impala::default.{table}').format('kudu').load()

In [21]:
store_in_kudu(df_mapas,'jogos')

In [23]:
table = read_from_kudu('jogos')
table.createOrReplaceTempView('jogos')
display(table)

DataFrame[partida: string, mapa: string, equipe1: string, equipe2: string, vitorioso: string, ct: string, tr: string]

In [24]:
condition = [df_limited.match_id == df2_limited.match_id, df_limited._map == df2_limited._map]
df_join = df_limited.join(df2_limited,condition,"inner").select(df_limited.match_id,df_limited._map, df_limited.team_1, df_limited.team_2,df_limited.match_winner,df2_limited.best_of)
display(df_join)

DataFrame[match_id: string, _map: string, team_1: string, team_2: string, match_winner: string, best_of: string]

In [27]:
df_missing_teams = df_join.select('team_2').distinct().subtract(df_join.select('team_1').distinct())
df_teams = df_join.select('team_1').distinct().union(df_missing_teams).withColumnRenamed('team_1','teams')
df_join.select('team_2').distinct().exceptAll(df_teams).show()
df_join.select('team_1').distinct().exceptAll(df_teams).show()

+------+
|team_2|
+------+
+------+

+------+
|team_1|
+------+
+------+



In [37]:
df_join = df_join.withColumnRenamed('match_id', 'partida') \
                    .withColumnRenamed('_map', 'mapa') \
                .withColumnRenamed('team_1', 'equipe1') \
                .withColumnRenamed('team_2', 'equipe2') \
                .withColumnRenamed('match_winner', 'vitorioso') \
                .withColumnRenamed('best_of', 'md')
display(df_join)

DataFrame[partida: string, mapa: string, equipe1: string, equipe2: string, vitorioso: string, md: string]

In [40]:
store_in_kudu(df_join,'proc')

In [41]:
teste = read_from_kudu('proc')
teste.createOrReplaceTempView('proc')
display(teste)

DataFrame[partida: string, mapa: string, equipe1: string, equipe2: string, vitorioso: string, md: string]

In [74]:
df_collect = df_teams.collect()
data = []
for i in range(df_teams.count()):
    selected_row = df_collect[i]
    current_team = selected_row['teams']
    
    num_jogos1 = spark.sql(f'SELECT COUNT(*) as cnt FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" ) a')
    num_jogos2 = spark.sql(f'SELECT COUNT(*) as cnt FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" ) a')
    total_jogos = num_jogos1.collect()[0][0] + num_jogos2.collect()[0][0]

    vitorias = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and vitorioso = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" and vitorioso = "2" ) a').collect()[0][0]
    derrotas = total_jogos - vitorias
    
    jmd5 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "5" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "5" ) a').collect()[0][0]
    md5 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "5" and vitorioso = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" and md = "5" and vitorioso = "2" ) a').collect()[0][0]

    jmd3 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "3" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "3" ) a').collect()[0][0]
    md3 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "3" and vitorioso = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" and md = "3" and vitorioso = "2" ) a').collect()[0][0]

    jmd2 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "2" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "2" ) a').collect()[0][0]
    md2 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "2" and vitorioso = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" and md = "2" and vitorioso = "2" ) a').collect()[0][0]

    jmd1 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "1" ) a').collect()[0][0]
    md1 = spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe1 = "{current_team}" and md = "1" and vitorioso = "1" ) a').collect()[0][0] + spark.sql(f'SELECT COUNT(*) as jogos FROM (SELECT DISTINCT partida FROM proc WHERE equipe2 = "{current_team}" and md = "1" and vitorioso = "2" ) a').collect()[0][0]

    data.append([current_team,total_jogos,vitorias,derrotas, md1, md2, md3, md5,jmd1, jmd2, jmd3,jmd5])

schema = ['equipe','jogos', 'vitorias', 'derrotas','md1', 'md2', 'md3', 'md5','jmd1', 'jmd2', 'jmd3','jmd5']
df_equipes = spark.createDataFrame(data, schema)
display(df_equipes)

6
29
4
83
7
1
8
6
70
30
3
49
5
21
13
2
7
59
33
41
4
64
22
11
21
6
14
102
12
19
5
9
30
103
37
146
3
10
32
292
2
1
16
1
1
31
9
16
56
25
11
1
116
179
10
20
30
2
3
2
27
34
5
19
3
19
136
10
11
3
14
4
3
6
1
20
166
7
24
44
17
89
3
1
86
2
6
1
1
1
1
6
10
1
14
23
12
8
2
11
3
16
2
2
2
22
16
63
11
12
5
8
13
2
6
8
2
9
6
10
41
1
7
14
2
15
1
2
6
7
33
372
3
3
67
2
2
7
51
2
18
35
12
8
1
1
85
1
52
42
3
29
1
6
61
32
2
26
4
4
26
119
3
1
7
3
17
2
5
5
4
4
9
2
11
2
20
28
6
260
34
8
1
4
398
1
2
7
8
1
67
2
15
13
2
9
27
1
2
2
99
29
185
1
3
92
37
15
24
50
2
19
6
9
1
46
1
1
363
146
6
2
134
31
17
13
15
69
23
5
2
8
2
397
29
2
2
2
14
5
355
34
194
9
3
72
13
7
25
9
49
1
28
291
3
1
2
6
1
3
12
20
154
50
382
130
2
133
46
16
9
28
2
10
79
8
148
5
19
1
193
5
14
39
73
4
2
1
35
225
4
23
12
364
13
26
4
4
6
4
12
8
4
15
2
7
4
2
175
2
74
41
7
17
27
6
4
326
146
26
36
64
7
93
12
1
28
23
56
31
12
17
5
6
338
1
7
23
1
339
7
191
8
35
2
7
4
24
31
12
25
2
1
12
5
22
11
11
5
373
4
82
7
2
7
20
3
27
7
2
5
17
376
28
46
34
12
179
44
8
4
97
5
2

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_equipes = df_equipes.withColumn('jogos', df_equipes.jogos.cast(DecimalType(8, 5))) \
    .withColumn('vitorias', df_equipes.vitorias.cast(DecimalType(8, 5))) \
    .withColumn('derrotas', df_equipes.derrotas.cast(DecimalType(8, 5))) \
    .withColumn('md1', df_equipes.md1.cast(DecimalType(8, 5))) \
    .withColumn('md2', df_equipes.md2.cast(DecimalType(8, 5))) \
    .withColumn('md3', df_equipes.md3.cast(DecimalType(8, 5))) \
    .withColumn('md5', df_equipes.md5.cast(DecimalType(8, 5))) \
    .withColumn('jmd1', df_equipes.jmd1.cast(DecimalType(8, 5))) \
    .withColumn('jmd2', df_equipes.jmd2.cast(DecimalType(8, 5))) \
    .withColumn('jmd3', df_equipes.jmd3.cast(DecimalType(8, 5))) \
    .withColumn('jmd5', df_equipes.md1.cast(DecimalType(8, 5)))

In [ ]:
store_in_kudu(df_equipes,'equipes')

# Data Pre Processing
This step will use the loaded data in Apache Kudu to pre process specific team win rates, and insert that data into de Neo4j database for later use during result prediction.

In [ ]:
# TODO - Pre process data in apache kudu and send to Neo4j

# Result Prediction
This is the main step to be used in result prediction, it will load data from Neo4j, while simultaneously processing raw generic data present in Apache Kudu. Results from both databases will then be fed into an algorithm that predicts the winner of the match.

In [ ]:
# TODO - Define functions that will interpret the match input and call multiple queries to fetch the win rates
# TODO - Define the algorithm that takes the win rates and outputs final prediction
# TODO - Define an interface for the user to call the interpreter function

# Bonus Queries
This section is used to call other queries not linked to result prediction like objective queries or player statistics.

In [ ]:
# low priority TODO 
# Define functions to execute objective queries in Neo4j
# Define functions to execute generic statistics queries in Apache Kudu
# Define interface to call above functions